In [1]:
# Cell 1: Environment Setup and Imports
import sys
import json
from pathlib import Path
import pandas as pd
from IPython.display import display

# Project root directory
PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == 'script' else Path.cwd()
print(f"Project root: {PROJECT_ROOT}")

# Add to path
sys.path.insert(0, str(PROJECT_ROOT))

# Import refactored step modules (with function interfaces)
from deeds_pipeline.step3_scraper import process_deeds_scraping
from deeds_pipeline.step4_geolocation import process_deeds_geolocation
from deeds_pipeline.step5_integration import process_deeds_integration

print("✓ All imports successful")
print(f"✓ nest_asyncio applied for Jupyter compatibility")

Project root: /Users/mingyang/Desktop/MCP/2025 Fall/11.458 Crowd Sourced City/deeds_pipeline
✓ All imports successful
✓ nest_asyncio applied for Jupyter compatibility


# Pipeline Step 1-2: Json reformatting and ORC extraction


In [ ]:
from deeds_pipeline.step1_json_reformat import run_step1

input_path = Path(
    "/Users/mingyang/Desktop/MCP/2025 Fall/11.458 Crowd Sourced City/deeds_pipeline/data/deed_reviews_northern_middlesex_20251103_110333.json")
output_path = Path(
    "/Users/mingyang/Desktop/MCP/2025 Fall/11.458 Crowd Sourced City/deeds_pipeline/output/step1_reformatted_by_deed_id.json")
# 使用默认配置运行
result = run_step1()

# 或指定自定义文件路径
result = run_step1(
    input_file=input_path,
    output_file=output_path
)

In [ ]:
output_path = Path(
    "/Users/mingyang/Desktop/MCP/2025 Fall/11.458 Crowd Sourced City/deeds_pipeline/output/step1_reformatted_by_deed_id.json")

# read the output file from step1
with open(output_path, 'r') as f:
    step1_data = json.load(f)

# print the first 3 records
print(step1_data[list(step1_data.keys())[3]])

## Step 2 Test: OCR Extraction (Testing with 3 Deeds)

Before running Step 2 on all 570 deeds, let's test with just 3 deeds to verify everything works correctly.

In [ ]:
# Step 2: OCR Extraction
from deeds_pipeline.step2_ocr_extraction import run_step2

# 运行 Step 2
result = run_step2()

# 访问特定契约的 OCR 结果
deed_1612 = result["1612"]
for ocr_result in deed_1612["ocr_results"]:
    print(f"Text: {ocr_result['ocr_text'][:100]}")
    print(f"Covenant: {ocr_result['covenant_detection']['covenant_detected']}")
    print(f"Addresses: {ocr_result['extracted_info']['street_addresses']}")

# Pipeline Step 3-5: Unified Deed Processing Notebook

This notebook implements a stable, function-based pipeline:
- **Step 3**: Scrape MassLand Records (using proven massland_scraper.py)
- **Step 4**: Geocode streets (with nest_asyncio for Jupyter compatibility)
- **Step 5**: Integrate and export data

## Key Features
- Function call interfaces (no global JSON files required)
- Optional JSON checkpointing between steps
- Fresh browser per deed (prevents session pollution)
- Async-compatible geolocation

## Input Format
List of deed records:
```python
[
    {"deed_id": "5767", "book": "57", "page": "21", "county": "Middlesex County", "town": "Dracut"},
    ...
]
```

In [ ]:
# Cell 2: Utility Functions
def load_json(filepath):
    """Load JSON file"""
    with open(filepath, 'r', encoding='utf-8') as f:
        return json.load(f)

def save_json(data, filepath):
    """Save data to JSON file"""
    Path(filepath).parent.mkdir(parents=True, exist_ok=True)
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=2, ensure_ascii=False)

def extract_book_page_from_urls(urls):
    """Extract book/page from MassLand URLs"""
    import re
    pairs = []
    for url in urls:
        match = re.search(r'[Bb]ook=(\d+).*?[Pp]age=(\d+)', url)
        if match:
            pairs.append({"book": match.group(1), "page": match.group(2)})
    return pairs

print("✓ Utility functions defined")

## Input Data: Choose Your Source

You can either:
1. **Option A**: Load from Step 2 output JSON file
2. **Option B**: Manually define test records

In [ ]:
# Cell 3A: OPTION A - Load from Step 2 Output

# Load Step 2 output
step2_file = PROJECT_ROOT / "output" / "step2_ocr_extracted.json"
step2_data = load_json(step2_file)

# Use one record per deed (scraper will extract book/page combos itself)
input_records = list(step2_data.values())

print(f"\n✓ Loaded {len(step2_data)} deeds from Step 2")
print(f"✓ Prepared {len(input_records)} deed records for scraping")
print(f"\nSample records:")
for rec in input_records[:3]:
    print(f"  - Deed {rec['deed_id']}: County {rec.get('county', '')}, Town: {rec.get('city', '')}")

In [8]:
# Cell 3B: OPTION B - Manual Input (Alternative)

# Uncomment and run this cell instead of 3A for manual testing
# input_records = [
#     {"deed_id": "5767", "book": "57", "page": "21", "county": "Middlesex County", "town": "Dracut"},
#     {"deed_id": "6188", "book": "61", "page": "88", "county": "Middlesex County", "town": "Dracut"}
# ]

# print("✓ Using manual input")
# print(f"✓ Total records: {len(input_records)}")
# for rec in input_records:
#     print(f"  - Deed {rec['deed_id']}: Book {rec['book']}, Page {rec['page']}")

## Step 3: Web Scraping

Scrape MassLand Records using the proven massland_scraper.py.
- Fresh browser created for each deed (prevents session pollution)
- Automatic retry and error handling
- Progress logging

In [ ]:
# Cell 4: Run Step 3 - Web Scraping

print("="*80)
print("STEP 3: SCRAPING MASSLAND RECORDS")
print("="*80)
print(f"\nInput: {len(input_records)} records")
print("Note: Browser will open for each deed (fresh session prevents errors)\n")

# Run Step 3 using function interface
step3_results = process_deeds_scraping(
    deed_records=input_records,
    headless=False  # Set to True to hide browser
)

# Optional: Save checkpoint
step3_checkpoint_file = PROJECT_ROOT / "output" / "notebook_step3_checkpoint.json"
save_json({r["deed_id"]: r for r in step3_results}, step3_checkpoint_file)

# Statistics
total_scraped = sum(1 for r in step3_results if r.get("step3_completed"))
total_streets = sum(len(r.get("extracted_streets", [])) for r in step3_results)

print(f"\n{'='*80}")
print(f"STEP 3 COMPLETED")
print(f"{'='*80}")
print(f"Checkpoint saved: {step3_checkpoint_file}")
print(f"Deeds processed: {total_scraped}/{len(step3_results)}")
print(f"Total unique streets: {total_streets}")
print(f"\nSample result:")
if step3_results:
    sample = step3_results[0]
    print(f"  Deed {sample['deed_id']}: {len(sample.get('extracted_streets', []))} streets found")
    print(f"  Streets: {', '.join(sample.get('extracted_streets', [])[:5])}")

## Step 4: Geolocation

Geocode streets using OpenStreetMap Nominatim API.
- Async HTTP requests for better performance
- Clustering validation to identify primary location
- Works seamlessly in Jupyter (nest_asyncio applied)

In [9]:
# Cell 5: Run Step 4 - Geolocation

print("="*80)
print("STEP 4: GEOCODING STREETS")
print("="*80)
print(f"\nInput: {len(step3_results)} deed records")
print("Using OpenStreetMap Nominatim API...\n")

# Run Step 4 using function interface (nest_asyncio handles event loop)
step4_results = process_deeds_geolocation(deed_records=step3_results)

# Optional: Save checkpoint
step4_checkpoint_file = PROJECT_ROOT / "output" / "notebook_step4_checkpoint.json"
save_json({r["deed_id"]: r for r in step4_results}, step4_checkpoint_file)

# Statistics
geocoded_count = sum(
    1 for r in step4_results 
    if r.get("geolocation") and r["geolocation"].get("cluster_center_lat")
)
avg_confidence = sum(
    r.get("geolocation", {}).get("confidence", 0) 
    for r in step4_results
    if r.get("geolocation")
) / max(geocoded_count, 1)

print(f"\n{'='*80}")
print(f"STEP 4 COMPLETED")
print(f"{'='*80}")
print(f"Checkpoint saved: {step4_checkpoint_file}")
print(f"Deeds geocoded: {geocoded_count}/{len(step4_results)}")
print(f"Average confidence: {avg_confidence:.1%}")
print(f"\nSample geolocation:")
for r in step4_results:
    if r.get("geolocation") and r["geolocation"].get("cluster_center_lat"):
        geo = r["geolocation"]
        print(f"  Deed {r['deed_id']}: ({geo['cluster_center_lat']:.4f}, {geo['cluster_center_lon']:.4f})")
        print(f"    Town: {geo.get('primary_town')}, Confidence: {geo.get('confidence', 0):.1%}")
        break

2025-12-06 15:13:18,135 - step4_geolocation - INFO - Event loop detected (likely Jupyter). Using nest_asyncio.
2025-12-06 15:13:18,137 - step4_geolocation - INFO - Starting Step 4 processing for 570 deed(s)
2025-12-06 15:13:18,137 - step4_geolocation - INFO - StreetClusteringValidator initialized (in-module implementation)
2025-12-06 15:13:18,138 - step4_geolocation - INFO - [1/570] Processing deed 1612
2025-12-06 15:13:18,138 - step4_geolocation - INFO - Deed 1612: Geocoding 10 streets in DRACUT, Northern Middlesex
2025-12-06 15:13:18,139 - step4_geolocation - INFO - Validating 10 streets in DRACUT, Northern Middlesex, Massachusetts


STEP 4: GEOCODING STREETS

Input: 570 deed records
Using OpenStreetMap Nominatim API...



2025-12-06 15:13:37,863 - step4_geolocation - INFO - Found 19 total candidates from 10 streets
2025-12-06 15:13:37,873 - step4_geolocation - INFO - Densest cluster: 19 candidates at (42.6869, -71.3652)
2025-12-06 15:13:37,874 - step4_geolocation - INFO - Town distribution: {'Collinsville': 6, 'Dracut': 10, 'Long Pond Park': 3}
2025-12-06 15:13:37,875 - step4_geolocation - INFO - Primary town identified: Dracut
2025-12-06 15:13:37,875 - step4_geolocation - INFO - Filtered: 9 streets in Dracut, 1 streets excluded
2025-12-06 15:13:39,059 - step4_geolocation - INFO - Final cluster: 9 streets, radius 0.51 mi, confidence 87.9%
2025-12-06 15:13:39,060 - step4_geolocation - INFO - [2/570] Processing deed 2168
2025-12-06 15:13:39,063 - step4_geolocation - INFO - Deed 2168: Geocoding 10 streets in DRACUT, Northern Middlesex
2025-12-06 15:13:39,065 - step4_geolocation - INFO - Validating 10 streets in DRACUT, Northern Middlesex, Massachusetts
2025-12-06 15:13:58,022 - step4_geolocation - INFO - F


STEP 4 COMPLETED
Checkpoint saved: /Users/mingyang/Desktop/MCP/2025 Fall/11.458 Crowd Sourced City/deeds_pipeline/output/notebook_step4_checkpoint.json
Deeds geocoded: 468/570
Average confidence: 85.6%

Sample geolocation:
  Deed 1612: (42.6874, -71.3653)
    Town: Dracut, Confidence: 87.9%


## Step 5: Data Integration

Integrate all processing results and export:
- Flatten nested data for CSV export
- Generate quality report
- Export to both JSON and CSV formats

In [22]:
# Cell 6: Run Step 5 - Integration and Export

print("="*80)
print("STEP 5: DATA INTEGRATION AND EXPORT")
print("="*80)
print(f"\nInput: {len(step4_results)} deed records")

# Run Step 5 using function interface
final_records, final_df, quality_report = process_deeds_integration(deed_records=step4_results)

# Save outputs
step5_json_file = PROJECT_ROOT / "output" / "notebook_final_output.json"
step5_csv_file = PROJECT_ROOT / "output" / "notebook_final_output.csv"

# Save JSON (full nested structure)
final_json_output = {
    "metadata": {
        "total_deeds": len(final_records),
        "quality_report": quality_report
    },
    "deeds": {r["deed_id"]: r for r in final_records}
}
save_json(final_json_output, step5_json_file)

# Save CSV (flattened structure)
final_df.to_csv(step5_csv_file, index=False, encoding='utf-8')

print(f"\n{'='*80}")
print(f"STEP 5 COMPLETED")
print(f"{'='*80}")
print(f"JSON output: {step5_json_file}")
print(f"CSV output: {step5_csv_file}")
print(f"\nQuality Report:")
for key, value in quality_report.items():
    print(f"  {key}: {value}")

2025-12-06 20:49:58,606 - step5_integration - INFO - Starting Step 5 processing for 570 deed(s)
2025-12-06 20:49:58,608 - step5_integration - INFO - Flattening deed records...
2025-12-06 20:49:58,612 - step5_integration - INFO - Flattened 570 records (unique deed_ids)
2025-12-06 20:49:58,613 - step5_integration - INFO - Generating quality report...
2025-12-06 20:49:58,614 - step5_integration - INFO - ============================================================
2025-12-06 20:49:58,614 - step5_integration - INFO - DATA QUALITY REPORT
2025-12-06 20:49:58,614 - step5_integration - INFO - ============================================================
2025-12-06 20:49:58,615 - step5_integration - INFO - total_deeds: 570
2025-12-06 20:49:58,615 - step5_integration - INFO - step2_ocr_completed: 570
2025-12-06 20:49:58,615 - step5_integration - INFO - step2_completion_rate: 100.0%
2025-12-06 20:49:58,615 - step5_integration - INFO - step3_scraper_completed: 570
2025-12-06 20:49:58,616 - step5_int

STEP 5: DATA INTEGRATION AND EXPORT

Input: 570 deed records

STEP 5 COMPLETED
JSON output: /Users/mingyang/Desktop/MCP/2025 Fall/11.458 Crowd Sourced City/deeds_pipeline/output/notebook_final_output.json
CSV output: /Users/mingyang/Desktop/MCP/2025 Fall/11.458 Crowd Sourced City/deeds_pipeline/output/notebook_final_output.csv

Quality Report:
  total_deeds: 570
  step2_ocr_completed: 570
  step2_completion_rate: 100.0%
  step3_scraper_completed: 570
  step3_completion_rate: 100.0%
  step4_geolocation_completed: 570
  step4_completion_rate: 100.0%
  original_covenant_count: 0
  ocr_detected_covenant_count: 0
  geocoded_count: 468
  geocoded_rate: 82.1%
  with_streets_count: 469
  with_streets_rate: 82.3%


## Results Visualization

View the final processed data and create visualizations.

In [23]:
# Cell 7: View and Analyze Results

print("="*80)
print("FINAL RESULTS SUMMARY")
print("="*80)
print(f"\nDataFrame shape: {final_df.shape}")
print(f"Columns: {list(final_df.columns)}")
print(f"\nFirst few rows:")
display(final_df.head())

# Geocoding statistics
geocoded_count = final_df['geo_latitude'].notna().sum()
avg_confidence = final_df['geo_confidence'].mean()

print(f"\nGeocoding Statistics:")
print(f"  Total deeds: {len(final_df)}")
print(f"  Successfully geocoded: {geocoded_count} ({geocoded_count/len(final_df)*100:.1f}%)")
if pd.notna(avg_confidence):
    print(f"  Average confidence: {avg_confidence:.1%}")

# Street extraction statistics
with_streets = final_df['scraped_streets'].notna().sum()
total_streets = final_df['scraped_street_count'].sum()
print(f"\nStreet Extraction:")
print(f"  Deeds with streets: {with_streets} ({with_streets/len(final_df)*100:.1f}%)")
print(f"  Total streets found: {int(total_streets)}")
print(f"  Average streets per deed: {total_streets/max(with_streets,1):.1f}")

FINAL RESULTS SUMMARY

DataFrame shape: (570, 30)
Columns: ['deed_id', 'review_ids', 'city', 'deed_date', 'address', 'is_restrictive_covenant', 'county', 'grantors', 'grantees', 'covenant_text', 'ocr_covenant_detected', 'ocr_covenant_text', 'plan_books', 'plan_pages', 'lot_numbers', 'extracted_streets', 'extracted_towns', 'scraped_streets', 'scraped_street_count', 'geo_latitude', 'geo_longitude', 'geo_address', 'geo_town', 'geo_cluster_radius_miles', 'geo_confidence', 'geo_validated_street_count', 'geo_invalid_street_count', 'step2_completed', 'step3_completed', 'step4_completed']

First few rows:


,deed_id,review_ids,city,deed_date,address,is_restrictive_covenant,county,grantors,grantees,covenant_text,...,geo_longitude,geo_address,geo_town,geo_cluster_radius_miles,geo_confidence,geo_validated_street_count,geo_invalid_street_count,step2_completed,step3_completed,step4_completed
0,1612,,None,1942-06-06,None,None,Northern Middlesex,"Fred E. Kroker, Ethel W. Kroker","Timothy F. Keane, Anna M. Keane",None,...,-71.365284,"99, Valley Road, Dracut, Middlesex County, Mas...",Dracut,0.511655,0.878835,9,1,True,True,True
1,2168,,None,1946-07-29,None,None,Northern Middlesex,"Jacob W. Lindley, Clara E Lindley","Max Ludwig, Jr. and Adelard Gagnon",None,...,-71.363107,"51, Valley Road, Dracut, Middlesex County, Mas...",Dracut,1.844340,0.815566,10,0,True,True,True
2,2153,,Dracut,1946-07-29,None,None,Northern Middlesex,"Sheldon E. Chase, Ella H. Chase",John Michaud,None,...,-71.363107,"51, Valley Road, Dracut, Middlesex County, Mas...",Dracut,1.844340,0.815566,10,0,True,True,True
3,2382,,None,1947-11-13,None,None,Northern Middlesex,"E. Gaston Campbell, Frank J. Rochette, Thomas ...","William C. Martin, Anna E. Martin",None,...,-71.363890,"271, Bouchard Avenue, Collinsville, Dracut, Mi...",Long Pond Park,0.657989,0.846701,7,1,True,True,True
4,2551,,Dracut,1955-12-01,None,None,Northern Middlesex,"George Beaulieu, Courana L. Beaulieu",Union National Bank of Lowell,None,...,-71.365284,"99, Valley Road, Dracut, Middlesex County, Mas...",Dracut,0.511655,0.878835,9,1,True,True,True



Geocoding Statistics:
  Total deeds: 570
  Successfully geocoded: 468 (82.1%)
  Average confidence: 85.4%

Street Extraction:
  Deeds with streets: 469 (82.3%)
  Total streets found: 4520
  Average streets per deed: 9.6


In [ ]:
# Cell 8: Create Interactive Map (Optional)

try:
    import folium
    from folium import plugins
    
    # Create base map
    center_lat = final_df['geo_latitude'].mean()
    center_lon = final_df['geo_longitude'].mean()
    
    if pd.notna(center_lat) and pd.notna(center_lon):
        m = folium.Map(location=[center_lat, center_lon], zoom_start=11)
        
        # Add markers for each geocoded deed
        for idx, row in final_df.iterrows():
            if pd.notna(row['geo_latitude']):
                confidence = row.get('geo_confidence', 0)
                color = 'green' if confidence > 0.7 else 'orange' if confidence > 0.4 else 'red'
                
                popup_html = f"""
                <b>Deed {row['deed_id']}</b><br>
                Town: {row.get('geo_town', 'N/A')}<br>
                Confidence: {confidence:.1%}<br>
                Streets: {row.get('scraped_street_count', 0)}<br>
                Radius: {row.get('geo_cluster_radius_miles', 0):.2f} mi
                """
                
                folium.CircleMarker(
                    location=[row['geo_latitude'], row['geo_longitude']],
                    radius=8,
                    popup=folium.Popup(popup_html, max_width=300),
                    color=color,
                    fill=True,
                    fillColor=color,
                    fillOpacity=0.6
                ).add_to(m)
        
        # Add legend
        legend_html = '''
        <div style="position: fixed; 
                    bottom: 50px; right: 50px; width: 200px; height: 120px; 
                    background-color: white; z-index:9999; font-size:14px;
                    border:2px solid grey; border-radius: 5px; padding: 10px">
        <b>Confidence Legend</b><br>
        <i class="fa fa-circle" style="color:green"></i> High (&gt;70%)<br>
        <i class="fa fa-circle" style="color:orange"></i> Medium (40-70%)<br>
        <i class="fa fa-circle" style="color:red"></i> Low (&lt;40%)
        </div>
        '''
        m.get_root().html.add_child(folium.Element(legend_html))
        
        # Save map
        map_file = PROJECT_ROOT / "output" / "deeds_map.html"
        m.save(str(map_file))
        print(f"✓ Interactive map saved to {map_file}")
        print(f"  Open in browser to view")
        
        # Display in notebook
        display(m)
    else:
        print("⚠ No geocoded data available for mapping")
        
except ImportError:
    print("⚠ folium not installed. Install with: pip install folium")
    print("  Skipping map visualization")
except Exception as e:
    print(f"⚠ Error creating map: {e}")

⚠ folium not installed. Install with: pip install folium
  Skipping map visualization


## Pipeline Complete!

### Summary
- **Step 3**: Web scraping completed with fresh browser instances
- **Step 4**: Geocoding completed with async compatibility
- **Step 5**: Data integrated and exported

### Outputs
- `notebook_final_output.json` - Full nested data structure
- `notebook_final_output.csv` - Flattened data for analysis
- `deeds_map.html` - Interactive map visualization
- Checkpoint files at each step for debugging

### Next Steps
1. Review quality report and identify any failed records
2. Analyze geocoding confidence scores
3. Use CSV for further analysis or database import

## Step 6: Validation & Accuracy Metrics
Compare pipeline-generated addresses against 111 manually verified ground truth addresses.

**Binary Metrics (Loose → Strict):**
1. **Town Match** - Does geo_town match manual city?
2. **Street Match** - Is ground truth street in scraped streets?
3. **Has Geolocation** - Has valid lat/lon?
4. **In Radius** - Is ground truth location within cluster radius?

In [24]:
# Step 6a: Preprocess Manual Addresses (One-time - Geocode ground truth addresses)
# This creates output/manual_addresses_geocoded.csv with lat/lon for each manual address
# Only needs to run once - skip if file already exists

import subprocess
import sys
from pathlib import Path

geocoded_file = PROJECT_ROOT / "output" / "manual_addresses_geocoded.csv"

if not geocoded_file.exists():
    print("Geocoding manual addresses (this takes ~2 minutes)...")
    script_path = str(PROJECT_ROOT / 'script' / 'preprocess_manual_geocoding.py')
    subprocess.run([sys.executable, script_path])
else:
    print(f"✓ Geocoded file already exists: {geocoded_file.name}")
    print("  Skipping preprocessing. Delete file to re-geocode.")

✓ Geocoded file already exists: manual_addresses_geocoded.csv
  Skipping preprocessing. Delete file to re-geocode.


In [25]:
# Step 6b: Run Validation with Binary Metrics
# Compares pipeline output against geocoded ground truth addresses

import subprocess
import sys

print("Running V3 Validation (Binary Metrics)...")
print("="*80)

script_path = str(PROJECT_ROOT / 'script' / 'validate_pipeline_accuracy.py')
subprocess.run([sys.executable, script_path])

Running V3 Validation (Binary Metrics)...
PIPELINE ACCURACY VALIDATION (V3 - Binary Metrics)

1. Loading data files...
   Loaded 111 manual addresses (with geocoding)
   Loaded 570 pipeline records

2. Validating pipeline results...
   Processed 111 deeds
   - 111 found in pipeline
   - 0 not in pipeline

3. Calculating accuracy metrics...
--------------------------------------------------------------------------------

   BINARY METRICS (based on 111 validated deeds):
   ----------------------------------------------------------------------
   1. Town Name Match:         85/111  =   76.6%
   2. Street Name Match:       72/111  =   64.9%
   3. Has Geolocation:         86/111  =   77.5%
   4. In Cluster Radius:       85/111  =   76.6%
   ----------------------------------------------------------------------
   OVERALL (all 4 match):      72/111  =   64.9%
   ----------------------------------------------------------------------

   DETAILED BREAKDOWN:
   - 0/4 metrics matched:   25 deed

CompletedProcess(args=['/Users/mingyang/Desktop/MCP/2025 Fall/11.458 Crowd Sourced City/deeds_pipeline/.venv/bin/python', '/Users/mingyang/Desktop/MCP/2025 Fall/11.458 Crowd Sourced City/deeds_pipeline/script/validate_pipeline_accuracy.py'], returncode=0)

In [26]:
# Step 6c: Display Validation Metrics Summary

import json
import pandas as pd
from IPython.display import display

# Load metrics
metrics_file = PROJECT_ROOT / "output" / "validation_metrics.json"
if metrics_file.exists():
    with open(metrics_file, 'r') as f:
        metrics = json.load(f)
    
    print("="*80)
    print("VALIDATION METRICS SUMMARY")
    print("="*80)
    
    # Create summary table
    summary_data = []
    for metric_name, metric_data in metrics.get("metrics", {}).items():
        summary_data.append({
            "Metric": metric_name.replace("_", " ").title(),
            "Matched": metric_data["count"],
            "Total": metric_data["total"],
            "Rate (%)": f"{metric_data['rate']:.1f}"
        })
    
    summary_df = pd.DataFrame(summary_data)
    display(summary_df)
    
    # Overall
    overall = metrics.get("overall", {})
    print(f"\nOVERALL (All 4 Match): {overall.get('all_match_count', 0)}/{metrics.get('deeds_in_pipeline', 0)} = {overall.get('all_match_rate', 0):.1f}%")
    
    # Load and show detailed results
    detailed_file = PROJECT_ROOT / "output" / "validation_results_detailed.csv"
    if detailed_file.exists():
        detailed_df = pd.read_csv(detailed_file)
        print(f"\nDetailed Results (first 10 validated deeds):")
        validated = detailed_df[detailed_df['status'] == 'VALIDATED'][['deed_id', 'ground_truth_street', 'pipeline_town', 'town_match', 'street_match', 'has_geolocation', 'in_radius', 'distance_miles']]
        display(validated.head(10))
else:
    print("Run validation first to generate metrics.")

VALIDATION METRICS SUMMARY


,Metric,Matched,Total,Rate (%)
0,Town Match,85,111,76.6
1,Street Match,72,111,64.9
2,Has Geolocation,86,111,77.5
3,In Radius,85,111,76.6



OVERALL (All 4 Match): 72/111 = 64.9%

Detailed Results (first 10 validated deeds):


,deed_id,ground_truth_street,pipeline_town,town_match,street_match,has_geolocation,in_radius,distance_miles
0,159,HILLCREST RD,Dracut,1,1,1,1,0.083
1,156,SHORE DR,Dracut,1,1,1,1,0.305
2,187,HILLTOP ROAD,Dracut,1,1,1,1,0.196
3,125,HILLTOP RD,Dracut,1,1,1,1,0.217
4,72,LONG DR,Dracut,1,1,1,1,0.155
5,118,HILLCREST RD,Dracut,1,1,1,1,0.292
6,96,HILLTOP RD,NaN,0,0,0,0,NaN
7,153,BROAD RD,NaN,0,0,0,0,NaN
8,115,HILLCREST RD,Dracut,1,1,1,1,0.138
9,110,HILLTOP RD,Dracut,1,1,1,1,0.153
